In [1]:
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
import jpx_tokyo_market_prediction
from sklearn.impute import SimpleImputer


from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

In [2]:
!pip install optuna

In [3]:
train_df = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv', parse_dates=True)
valid_df = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv', parse_dates=True)
test_df = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/example_test_files/stock_prices.csv', parse_dates=True)

In [4]:
valid_df.isnull()
#valid_df.isnull().sum().sum()
valid_df.dropna() #replaced valid_df.dropna(inplace=True) to only valid_df.dropna()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
14353,20211215_3191,2021-12-15,3191,1455.0,1494.0,1455.0,1490.0,164300,1.0,21.0,False,-0.023904
15510,20211215_7962,2021-12-15,7962,887.0,889.0,881.0,885.0,14000,1.0,7.0,False,-0.008919
15822,20211215_9267,2021-12-15,9267,4775.0,4920.0,4755.0,4860.0,117500,1.0,12.5,False,0.000000
30007,20211227_1381,2021-12-27,1381,3150.0,3170.0,3120.0,3155.0,6500,1.0,0.0,False,0.015873
30009,20211227_1413,2021-12-27,1413,2254.0,2254.0,2228.0,2244.0,16600,1.0,50.0,False,-0.040404
...,...,...,...,...,...,...,...,...,...,...,...,...
227720,20220526_8923,2022-05-26,8923,1121.0,1137.0,1118.0,1127.0,85400,1.0,0.0,False,0.024845
227787,20220526_9278,2022-05-26,9278,1000.0,1006.0,999.0,999.0,165800,1.0,20.0,False,-0.012961
227800,20220526_9369,2022-05-26,9369,1079.0,1096.0,1077.0,1085.0,20500,1.0,11.5,False,-0.002757
227886,20220526_9717,2022-05-26,9717,1150.0,1150.0,1134.0,1141.0,15400,1.0,0.0,False,0.020158


In [5]:
print(train_df.shape )
print(valid_df.shape )


print("train_df Null Check : {}".format(train_df.isnull().sum()))
print('----------------------------------------------------------')
print("valid_df Null Check : {}".format(valid_df.isnull().sum()))


(2332531, 12)
(269881, 12)
train_df Null Check : RowId                     0
Date                      0
SecuritiesCode            0
Open                   7608
High                   7608
Low                    7608
Close                  7608
Volume                    0
AdjustmentFactor          0
ExpectedDividend    2313666
SupervisionFlag           0
Target                  238
dtype: int64
----------------------------------------------------------
valid_df Null Check : RowId                    0
Date                     0
SecuritiesCode           0
Open                   818
High                   818
Low                    818
Close                  818
Volume                   0
AdjustmentFactor         0
ExpectedDividend    267870
SupervisionFlag          0
Target                   8
dtype: int64


In [6]:
prices = valid_df

del train_df, valid_df

In [7]:
prices = prices.drop(["ExpectedDividend"],axis=1) #ExpectedDividend is emty column.

In [8]:
#prices['Open'].fillna(value=prices['Open'].mean(), inplace=True)
#prices['High'].fillna(value=prices['High'].mean(), inplace=True)
#prices['Low'].fillna(value=prices['Low'].mean(), inplace=True)
#prices['Close'].fillna(value=prices['Close'].mean(), inplace=True)
#prices['Target'].fillna(value=prices['Target'].mean(), inplace=True)
# Result with above change produce accuracy of .99916
imputer = SimpleImputer(strategy='mean',
                        missing_values=np.nan)    #strategy='constant',missing_values=np.nan, fill_value=0 produced score of 5.427 with an accuracy of .999993
imputer = imputer.fit(prices[['Open','High','Low','Close','Target']])
prices[['Open','High','Low','Close','Target']] = imputer.transform(prices[['Open','High','Low','Close','Target']])      #prices.iloc[:,:] = imputer.transform(prices)
#prices.dropna(inplace=True)

In [9]:
prices.head(35)

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,SupervisionFlag,Target
0,20211206_1301,2021-12-06,1301,2982.000000,2982.00000,2965.000000,2971.000000,8900,1.0,False,-0.003263
1,20211206_1332,2021-12-06,1332,592.000000,599.00000,588.000000,589.000000,1360800,1.0,False,-0.008993
2,20211206_1333,2021-12-06,1333,2368.000000,2388.00000,2360.000000,2377.000000,125900,1.0,False,-0.009963
3,20211206_1375,2021-12-06,1375,1230.000000,1239.00000,1224.000000,1224.000000,81100,1.0,False,-0.015032
4,20211206_1376,2021-12-06,1376,1339.000000,1372.00000,1339.000000,1351.000000,6200,1.0,False,0.002867
5,20211206_1377,2021-12-06,1377,3185.000000,3215.00000,3180.000000,3200.000000,77000,1.0,False,0.003044
6,20211206_1379,2021-12-06,1379,1890.000000,1898.00000,1885.000000,1889.000000,48500,1.0,False,-0.009870
7,20211206_1381,2021-12-06,1381,3200.000000,3200.00000,3150.000000,3150.000000,1600,1.0,False,0.000000
8,20211206_1407,2021-12-06,1407,6100.000000,6290.00000,6010.000000,6220.000000,246700,1.0,False,-0.004732
9,20211206_1413,2021-12-06,1413,2185.000000,2211.00000,2181.000000,2190.000000,9200,1.0,False,0.002222


In [10]:
average = pd.DataFrame(prices.groupby("SecuritiesCode").Target.mean())
def get_avg(_id_):
    return average.loc[_id_]



def getadvance(x):
    ret = 0
    if x > 0:
        ret = 1
    return(ret)

def get_month(dt):
    x = dt.strftime("%m")
    return(x)


def cat_col(data) :
    data['SecuritiesCode'] = data['SecuritiesCode'].astype('category')
    data['SupervisionFlag'] = data['SupervisionFlag'].astype('category')
    data['advance'] = data['advance'].astype('category')
    data['AdjustmentFactor'] = data['AdjustmentFactor'].astype('category')
    data['Month'] = data['Month'].astype('category')
    return data 

In [11]:
prices["Avg"] = prices["SecuritiesCode"].apply(get_avg)


prices['pClose'] = prices['Close'].shift(-1)
prices['delta'] = prices['Close'] - prices['pClose']
prices['advance'] = list(map(getadvance, prices['delta']))
prices['Date'] = pd.to_datetime(prices['Date'], format = "%Y-%m-%d")
prices['Month'] =  list(map(get_month, prices['Date']))

prices = cat_col(prices)

In [12]:
prices.isnull().sum()

RowId               0
Date                0
SecuritiesCode      0
Open                0
High                0
Low                 0
Close               0
Volume              0
AdjustmentFactor    0
SupervisionFlag     0
Target              0
Avg                 0
pClose              1
delta               1
advance             0
Month               0
dtype: int64

In [13]:
prices.columns

Index(['RowId', 'Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close',
       'Volume', 'AdjustmentFactor', 'SupervisionFlag', 'Target', 'Avg',
       'pClose', 'delta', 'advance', 'Month'],
      dtype='object')

In [14]:
features = ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close',
       'Volume', 'AdjustmentFactor', 'SupervisionFlag', 'Avg',
       'pClose', 'delta', 'advance', 'Month']

In [15]:
# data segmentation
prices['Date'] = prices['Date'].dt.strftime("%Y%m%d").astype(int)

X = prices[features]
y = prices['Target']
codes = X.SecuritiesCode.unique()

In [16]:
def objectives(trial):
    params = {
            'num_leaves': trial.suggest_int('num_leaves', 1000, 6000),
            'n_estimators': trial.suggest_int('n_estimators', 1000, 6000),
            'max_bin': trial.suggest_int('max_bin', 200, 400),
            'learning_rate': trial.suggest_uniform('learning_rate',0, 0.9),
    }

    model = LGBMRegressor(**params)
    model.fit(X,y)
    score = model.score(X,y)
    return score

In [17]:
import optuna

studyLGBM = optuna.create_study(direction='maximize',sampler=optuna.samplers.RandomSampler(seed=0))
studyLGBM.optimize(objectives, n_trials=3)

trial = studyLGBM.best_trial
params_best = dict(trial.params.items())
params_best['random_seed'] = 0
    
model = LGBMRegressor(**params_best)

[I 2022-07-05 17:00:18,270] A new study created in memory with name: no-name-2bb66f59-d5fd-4149-9663-ccbd15bdc19a
[I 2022-07-05 17:54:16,148] Trial 0 finished with value: 0.9999999999999993 and parameters: {'num_leaves': 3744, 'n_estimators': 4576, 'max_bin': 321, 'learning_rate': 0.49039486469720717}. Best is trial 0 with value: 0.9999999999999993.
[I 2022-07-05 18:31:53,074] Trial 1 finished with value: 0.9999999999999993 and parameters: {'num_leaves': 3118, 'n_estimators': 4230, 'max_bin': 287, 'learning_rate': 0.8025957007038718}. Best is trial 0 with value: 0.9999999999999993.
[I 2022-07-05 19:19:50,745] Trial 2 finished with value: 0.9999999998369631 and parameters: {'num_leaves': 5819, 'n_estimators': 2917, 'max_bin': 359, 'learning_rate': 0.47600542777761407}. Best is trial 0 with value: 0.9999999999999993.


In [18]:
print('study.best_params:', studyLGBM.best_trial.value)
print('Number of finished trials:', len(studyLGBM.trials))
print('Best trial:', studyLGBM.best_trial.params)
print('study.best_params:', studyLGBM.best_params)

study.best_params: 0.9999999999999993
Number of finished trials: 3
Best trial: {'num_leaves': 3744, 'n_estimators': 4576, 'max_bin': 321, 'learning_rate': 0.49039486469720717}
study.best_params: {'num_leaves': 3744, 'n_estimators': 4576, 'max_bin': 321, 'learning_rate': 0.49039486469720717}


In [19]:
#Parameter importance Visual
#optuna.visualization.plot_param_importances(studyLGBM)

In [20]:
model.fit(X,y)
model.score(X,y)

0.9999999999999993

In [21]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    #ds=[prices, options, financials, trades, secondary_prices, sample_prediction]
        
    prices["Avg"] = sample_prediction["SecuritiesCode"].apply(get_avg)
    
    prices['pClose'] = prices['Close'].shift(-1)
    prices['delta'] = prices['Close'] - prices['pClose']
    prices['advance'] = list(map(getadvance, prices['delta']))
    prices['Date'] = pd.to_datetime(prices['Date'], format = "%Y-%m-%d")
    prices['Month'] =  list(map(get_month, prices['Date']))
    

    prices = cat_col(prices)
    prices['Date'] = prices['Date'].dt.strftime("%Y%m%d").astype(int)
    print('-------------------------------prices------------------------------')
    print(prices)
    print('------------------------------------------------------------------------------')    
    
    prices = prices.drop(['RowId','ExpectedDividend'],axis=1)

    sample_prediction["Prediction"] = model.predict(prices)
    print('-------sample_prediction--------')
    print(sample_prediction)
    sample_prediction = sample_prediction.sort_values(by = "Prediction", ascending=False)
    sample_prediction.Rank = np.arange(0,2000)
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    sample_prediction.drop(["Prediction"],axis=1)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    print('-------------------------------submission------------------------------')
    print(submission)
    print('------------------------------------------------------------------------------')
    env.predict(submission)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
-------------------------------prices------------------------------
          Date          RowId SecuritiesCode    Open    High     Low   Close  \
0     20211206  20211206_1301           1301  2982.0  2982.0  2965.0  2971.0   
1     20211206  20211206_1332           1332   592.0   599.0   588.0   589.0   
2     20211206  20211206_1333           1333  2368.0  2388.0  2360.0  2377.0   
3     20211206  20211206_1375           1375  1230.0  1239.0  1224.0  1224.0   
4     20211206  20211206_1376           1376  1339.0  1372.0  1339.0  1351.0   
...        ...            ...            ...     ...     ...     ...     ...   
1995  20211206  20211206_9990           9990   528.0   531.0   516.0   517.0   
1996  20211206  20211206_9991           9991   796.0   800.0   785.0   785.0   
1997  20211206  20211206_9993           9993  1645.0  1653.0  1627.0  1627.0   
1998  2